## Naive Bayes Approach

In [163]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [164]:
os.chdir('/Users/meldrumapple/Desktop/Bayes/Final')

In [165]:
#import test and train data set as a result of Data_engineering code
test=pd.read_csv('test.csv')
train=pd.read_csv('train.csv')

In [166]:
# adjust score range to start at zero for varaibles age, income, education for ease of code later
train['Age']=train['Age']-1
train['Income']=train['Income']-1
train['Education']=train['Education']-1
train['GenHlth']=train['GenHlth']-1
test['Age']=test['Age']-1
test['Income']=test['Income']-1
test['Education']=test['Education']-1
test['GenHlth']=test['GenHlth']-1

In [167]:
#split train data set by response variable
train_0=train[train['Diabetes_012']==0]
train_1=train[train['Diabetes_012']==1]
train_2=train[train['Diabetes_012']==2]

likelihood of any given combination of variable reponses: 
$$
p(X|c_i)=\prod_{j=0}^{J}P(x_j|c_i)
$$

In [168]:
#Function that gets likelihood of every combination of class and variable reponse for any variable
def get_like(variable,responses):
    like_0=list()
    like_1=list()
    like_2=list()
    for x in responses: 
        like_0.append(len(train_0[train_0[variable]==x])/len(train_0))
        like_1.append(len(train_1[train_1[variable]==x])/len(train_1))
        like_2.append(len(train_2[train_2[variable]==x])/len(train_2))
    return pd.DataFrame({'like_0':like_0, 'like_1':like_1, 'like_2':like_2})

In [169]:
# get dictionary of dataframe for each variable
likes=list()
#add binary variable dataframes to dict
likes.append(get_like('HighBP',[0,1]))
likes.append(get_like('HighChol',[0,1]))
likes.append(get_like('CholCheck',[0,1]))
likes.append(get_like('BMI', list(range(0,5))))
likes.append(get_like('Smoker',[0,1]))
likes.append(get_like('Stroke',[0,1]))
likes.append(get_like('HeartDiseaseorAttack',[0,1]))
likes.append(get_like('PhysActivity',[0,1]))
likes.append(get_like('Fruits',[0,1]))
likes.append(get_like('Veggies',[0,1]))
likes.append(get_like('HvyAlcoholConsump',[0,1]))
likes.append(get_like('AnyHealthcare',[0,1]))
likes.append(get_like('GenHlth',[0,1]))
likes.append(get_like('GenHlth', list(range(0,6))))
likes.append(get_like('MentHlth', list(range(0,3))))
likes.append(get_like('PhysHlth', list(range(0,3))))
likes.append(get_like('DiffWalk',[0,1]))
likes.append(get_like('Sex',[0,1]))
likes.append(get_like('Age', list(range(0,14))))
likes.append(get_like('Education', list(range(0,7))))
likes.append(get_like('Income', list(range(0,9))))

Priors:
$$
P(c_j)
$$

In [183]:
#Get the priors
pri_0=len(train_0)/len(train)
pri_1=len(train_1)/len(train)
pri_2=len(train_2)/len(train)

condition for classification:
$$
argmax(P(c_j)[\prod_{j=0}^{J}P(x_j|c_i)+\alpha])
$$

we added a laplace smoothing factor because some x_i responses are not present in class c_j so they would result in a likelihood of zero which would give a posterior probability of zero for that class

In [187]:
def class_obs(row):
    like_0=1
    like_1=1
    like_2=1
    for i in range(len(row)):
        like_0*=likes[i].iloc[int(row[i]),0].round(15)+0.001 #alpha in case frequency is zero for any variable
        like_1*=likes[i].iloc[int(row[i]),1].round(15)+0.001
        like_2*=likes[i].iloc[int(row[i]),2].round(15)+0.001
    post_0=like_0*(pri_0)
    post_1=like_1*(pri_1)
    post_2=like_2*(pri_2)
    dct={post_0:0,post_1:1,post_2:2}
    return dct[max(post_0,post_1,post_2)]

In [188]:
# now apply to test data:
true_val=test.iloc[:,2].astype(int)
predictors=test.iloc[:,3:].astype(int)

In [189]:
# loop through all rows of the prediction variables and record most likely classfication
preds=list()
for j in range(len(test)):
    preds.append(class_obs(list(predictors.iloc[j, :])))

In [190]:
results=pd.DataFrame({'dia_hat':preds, 'dia':true_val})
results['correct']=(results['dia_hat']==results['dia'])*1

In [193]:
sum(results['correct'])/len(results)

0.8393054241564175